In [324]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
from itertools import islice, cycle
from more_itertools import pairwise
from tqdm.auto import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
os.chdir("C:\\Users\\Администратор.WIN-U1NLG8MM702\\Documents\\Katerina\\Datasets for analysis")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [325]:
sample = pd.read_csv('sample_submission.csv')
interactions = pd.read_csv('interactions.csv')
items = pd.read_csv('items.csv')
users = pd.read_csv('users.csv')

In [326]:
interactions.shape, items.shape, users.shape

((1562617, 5), (63758, 5), (137254, 3))

### Предобработка

In [327]:
interactions['start_date'] = pd.to_datetime(interactions['start_date'])

In [328]:
duplicates = interactions.duplicated(subset=['user_id', 'item_id'], keep=False)
df_duplicates = interactions[duplicates].sort_values(by=['user_id', 'start_date'])
interactions = interactions[~duplicates]

In [329]:
df_duplicates = df_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max',
    'rating': 'max',
    'start_date': 'min'
})
interactions = interactions.append(df_duplicates.reset_index(), ignore_index=True)

In [330]:
interactions['progress'] = interactions['progress'].astype(np.int8)
interactions['rating'] = interactions['rating'].astype(pd.SparseDtype(np.float32, np.nan))

In [331]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562505 entries, 0 to 1562504
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype               
---  ------      --------------    -----               
 0   user_id     1562505 non-null  int64               
 1   item_id     1562505 non-null  int64               
 2   progress    1562505 non-null  int8                
 3   rating      323563 non-null   Sparse[float32, nan]
 4   start_date  1562505 non-null  datetime64[ns]      
dtypes: Sparse[float32, nan](1), datetime64[ns](1), int64(2), int8(1)
memory usage: 39.7 MB


In [332]:
interactions.to_pickle('interactions_preprocessed.pickle')

In [333]:
users.head()

,user_id,age,sex
0,0,45_54,1.0
1,1,25_34,0.0
2,2,45_54,0.0
3,3,65_inf,0.0
4,4,18_24,0.0


In [334]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  137254 non-null  int64  
 1   age      137244 non-null  object 
 2   sex      135640 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.1+ MB


In [335]:
users.nunique()

user_id    137254
age             6
sex             2
dtype: int64

In [336]:
users['age'] = users['age'].astype('category')
users['sex'] = users['sex'].astype(pd.SparseDtype(np.float32, np.nan))


In [337]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype               
---  ------   --------------   -----               
 0   user_id  137254 non-null  int64               
 1   age      137244 non-null  category            
 2   sex      135640 non-null  Sparse[float32, nan]
dtypes: Sparse[float32, nan](1), category(1), int64(1)
memory usage: 2.2 MB


In [338]:
interaction_users = interactions['user_id'].unique()

common_users = len(np.intersect1d(interaction_users, users['user_id']))
users_only_in_interaction = len(np.setdiff1d(interaction_users, users['user_id']))
users_only_features = len(np.setdiff1d(users['user_id'], interaction_users))
total_users = common_users + users_only_in_interaction + users_only_features
print(f'Кол-во пользователей - {total_users}')
print(f'Кол-во пользователей c взаимодействиями и фичами - {common_users} ({common_users / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c взаимодействиями - {users_only_in_interaction} ({users_only_in_interaction / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c фичами - {users_only_features} ({users_only_features / total_users * 100:.2f}%)')

Кол-во пользователей - 164771
Кол-во пользователей c взаимодействиями и фичами - 130808 (79.39%)
Кол-во пользователей только c взаимодействиями - 27517 (16.70%)
Кол-во пользователей только c фичами - 6446 (3.91%)


In [339]:
users.to_pickle('users_preprocessed.pickle')

In [340]:
items.head()

,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,281311,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,213473,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [341]:
items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       63758 non-null  int64 
 1   title    63758 non-null  object
 2   genres   63753 non-null  object
 3   authors  56700 non-null  object
 4   year     49508 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


In [342]:
items.nunique()
#Почему колонка year типа object, а не int?

id         63758
title      58093
genres     11091
authors    17024
year        1074
dtype: int64

In [343]:
for col in ['genres', 'authors', 'year']:
    items[col] = items[col].astype('category')

In [344]:
items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   id       63758 non-null  int64   
 1   title    63758 non-null  object  
 2   genres   63753 non-null  category
 3   authors  56700 non-null  category
 4   year     49508 non-null  category
dtypes: category(3), int64(1), object(1)
memory usage: 2.5+ MB


In [345]:
interaction_items = interactions['item_id'].unique()

common_items = len(np.intersect1d(interaction_items, items['id']))
items_only_in_interaction = len(np.setdiff1d(interaction_items, items['id']))
items_only_features = len(np.setdiff1d(items['id'], interaction_items))
total_items = common_items + items_only_in_interaction + items_only_features
print(f'Кол-во книг - {total_items}')
print(f'Кол-во книг c взаимодействиями и фичами - {common_items} ({common_items / total_items * 100:.2f}%)')
print(f'Кол-во книг только c взаимодействиями - {items_only_in_interaction} ({items_only_in_interaction / total_items * 100:.2f}%)')
print(f'Кол-во книг только c фичами - {items_only_features} ({items_only_features / total_items * 100:.2f}%)')

Кол-во книг - 63758
Кол-во книг c взаимодействиями и фичами - 63758 (100.00%)
Кол-во книг только c взаимодействиями - 0 (0.00%)
Кол-во книг только c фичами - 0 (0.00%)


In [346]:
items.to_pickle('items_preprocessed.pickle')

In [347]:
users_inv_mapping = dict(enumerate(interactions['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)

158325

In [348]:
items_inv_mapping = dict(enumerate(interactions['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(items_mapping)

63758

In [349]:
items['title'] = items['title'].str.lower()

In [350]:
item_titles = pd.Series(items['title'].values, index=items['id']).to_dict()

In [351]:
title_items = items.groupby('title')['id'].agg(list)

In [352]:
title_count = title_items.map(len)
title_count.value_counts()

1     53608
2      3623
3       488
4       151
5        61
6        23
7        17
8         8
9         5
18        1
10        1
11        1
12        1
35        1
13        1
15        1
51        1
Name: id, dtype: int64

In [353]:
title_items[title_count > 1].tail()

title
яр                           [108212, 252893]
ярмарка тщеславия    [323949, 351537, 324231]
ярослав мудрый                [358342, 83942]
ярость тьмы                   [324469, 74554]
ящик пандоры                  [93562, 164956]
Name: id, dtype: object

In [354]:
items[items['title'] == 'ящик пандоры']

,id,title,genres,authors,year
39115,93562,ящик пандоры,"Любовно-фантастические романы,Научная фантастика",Филипп Хорват,2017
54893,164956,ящик пандоры,"Мистика,Современная зарубежная литература",Бернар Вербер,2018


In [355]:
items[items['title'] == '451 градус по фаренгейту']

,id,title,genres,authors,year
49284,99550,451 градус по фаренгейту,"Социальная фантастика,Зарубежная фантастика,На...",Рэй Брэдбери,"1951, 1953, 1967"


In [356]:
interactions['rating'] = np.array(interactions['rating'].values, dtype=np.float32)

interactions.loc[interactions['item_id'].isin([99550])].groupby('item_id').agg({
    'progress': np.size,
    'rating': ['mean'],
    'start_date': ['min', 'max'],
})

progress rating start_date           
            size   mean        min        max
item_id                                      
99550         55    3.4 2018-01-26 2019-12-30

### Validation

In [357]:
class TimeRangeSplit():
    """
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html
    """
    def __init__(self, 
                 start_date, 
                 end_date=None, 
                 freq='D', 
                 periods=None, 
                 tz=None, 
                 normalize=False, 
                 closed=None, 
                 train_min_date=None,
                 filter_cold_users=True, 
                 filter_cold_items=True, 
                 filter_already_seen=True):
        
        self.start_date = start_date
        if end_date is None and periods is None:
            raise ValueError("Either 'end_date' or 'periods' must be non-zero, not both at the same time.")

        self.end_date = end_date
        self.freq = freq
        self.periods = periods
        self.tz = tz
        self.normalize = normalize
        self.closed = closed
        self.train_min_date = pd.to_datetime(train_min_date, errors='raise')
        self.filter_cold_users = filter_cold_users
        self.filter_cold_items = filter_cold_items
        self.filter_already_seen = filter_already_seen

        self.date_range = pd.date_range(
            start=start_date, 
            end=end_date, 
            freq=freq, 
            periods=periods, 
            tz=tz, 
            normalize=normalize, 
            closed=closed)

        self.max_n_splits = max(0, len(self.date_range) - 1)
        if self.max_n_splits == 0:
            raise ValueError("Provided parametrs set an empty date range.") 

    def split(self, 
              df, 
              user_column='user_id',
              item_column='item_id',
              datetime_column='date',
              fold_stats=False):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            train_min_mask = df_datetime >= self.train_min_date
        else:
            train_min_mask = df_datetime.notnull()

        date_range = self.date_range[(self.date_range >= df_datetime.min()) & 
                                     (self.date_range <= df_datetime.max())]

        for start, end in pairwise(date_range):
            fold_info = {
                'Start date': start,
                'End date': end
            }
            train_mask = train_min_mask & (df_datetime < start)
            train_idx = df.index[train_mask]
            if fold_stats:
                fold_info['Train'] = len(train_idx)

            test_mask = (df_datetime >= start) & (df_datetime < end)
            test_idx = df.index[test_mask]
            
            if self.filter_cold_users:
                new = np.setdiff1d(
                    df.loc[test_idx, user_column].unique(), 
                    df.loc[train_idx, user_column].unique())
                new_idx = df.index[test_mask & df[user_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New users'] = len(new)
                    fold_info['New users interactions'] = len(new_idx)

            if self.filter_cold_items:
                new = np.setdiff1d(
                    df.loc[test_idx, item_column].unique(), 
                    df.loc[train_idx, item_column].unique())
                new_idx = df.index[test_mask & df[item_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New items'] = len(new)
                    fold_info['New items interactions'] = len(new_idx)

            if self.filter_already_seen:
                user_item = [user_column, item_column]
                train_pairs = df.loc[train_idx, user_item].set_index(user_item).index
                test_pairs = df.loc[test_idx, user_item].set_index(user_item).index
                intersection = train_pairs.intersection(test_pairs)
                test_idx = test_idx[~test_pairs.isin(intersection)]
                # test_mask = rd.df.index.isin(test_idx)
                if fold_stats:
                    fold_info['Known interactions'] = len(intersection)

            if fold_stats:
                fold_info['Test'] = len(test_idx)

            yield (train_idx, test_idx, fold_info)

    def get_n_splits(self, df, datetime_column='date'):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            df_datetime = df_datetime[df_datetime >= self.train_min_date]

        date_range = self.date_range[(self.date_range >= df_datetime.min()) & 
                                     (self.date_range <= df_datetime.max())]

        return max(0, len(date_range) - 1)

In [358]:
last_date = interactions['start_date'].max().normalize()
folds = 7
start_date = last_date - pd.Timedelta(days=folds)
start_date, last_date

(Timestamp('2019-12-24 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [359]:
cv = TimeRangeSplit(start_date=start_date, periods=folds+1)

cv.max_n_splits, cv.get_n_splits(interactions, datetime_column='start_date')

(7, 7)

In [360]:
cv.date_range

DatetimeIndex(['2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27',
               '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', freq='D')

In [361]:
folds_with_stats = list(cv.split(
    interactions, 
    user_column='user_id',
    item_column='item_id',
    datetime_column='start_date',
    fold_stats=True
))

folds_info_with_stats = pd.DataFrame([info for _, _, info in folds_with_stats])

In [362]:
folds_info_with_stats

,Start date,End date,Train,New users,New users interactions,New items,New items interactions,Known interactions,Test
0,2019-12-24,2019-12-25,1545728,3,3,0,0,0,2075
1,2019-12-25,2019-12-26,1547806,5,5,0,0,0,2102
2,2019-12-26,2019-12-27,1549913,3,5,0,0,0,2120
3,2019-12-27,2019-12-28,1552038,2,3,0,0,0,2096
4,2019-12-28,2019-12-29,1554137,2,2,0,0,0,2103
5,2019-12-29,2019-12-30,1556242,3,3,0,0,0,2051
6,2019-12-30,2019-12-31,1558296,3,3,0,0,0,2132


In [363]:
fold_dates = [(info['Start date'], info['End date']) for _, _, info in folds_with_stats]
fold_dates

[(Timestamp('2019-12-24 00:00:00', freq='D'),
  Timestamp('2019-12-25 00:00:00', freq='D')),
 (Timestamp('2019-12-25 00:00:00', freq='D'),
  Timestamp('2019-12-26 00:00:00', freq='D')),
 (Timestamp('2019-12-26 00:00:00', freq='D'),
  Timestamp('2019-12-27 00:00:00', freq='D')),
 (Timestamp('2019-12-27 00:00:00', freq='D'),
  Timestamp('2019-12-28 00:00:00', freq='D')),
 (Timestamp('2019-12-28 00:00:00', freq='D'),
  Timestamp('2019-12-29 00:00:00', freq='D')),
 (Timestamp('2019-12-29 00:00:00', freq='D'),
  Timestamp('2019-12-30 00:00:00', freq='D')),
 (Timestamp('2019-12-30 00:00:00', freq='D'),
  Timestamp('2019-12-31 00:00:00', freq='D'))]

In [364]:
def compute_metrics(df_true, df_pred, top_N):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    
    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
        result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    result[f'MRR'] = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    return pd.Series(result)

### LightFM

In [365]:
from lightfm.data import Dataset
from lightfm import LightFM


In [366]:
dataset = Dataset()
dataset.fit(interactions['user_id'].unique(),interactions['item_id'].unique())


In [367]:
users['age'] = users['age'].cat.add_categories('age_unknown')
users['age'] = users['age'].fillna('age_unknown')
age_features = users['age'].unique()
age_features

['45_54', '25_34', '65_inf', '18_24', '35_44', '55_64', 'age_unknown']
Categories (7, object): ['45_54', '25_34', '65_inf', '18_24', '35_44', '55_64', 'age_unknown']

In [368]:
users['sex'] = np.array(users['sex'].astype(str))
users['sex'] = users['sex'].fillna('age_unknown')
sex_features = users['sex'].unique()
sex_features

array(['1.0', '0.0', 'nan'], dtype=object)

In [369]:
users_features = np.append(age_features, sex_features)
users_features

array(['45_54', '25_34', '65_inf', '18_24', '35_44', '55_64',
       'age_unknown', '1.0', '0.0', 'nan'], dtype=object)

In [370]:
dataset.fit_partial(user_features=users_features)

In [371]:
items['genres'] = items['genres'].cat.add_categories('genre_unknown')
items['genres'] = items['genres'].fillna('genre_unknown')
genres = list(items['genres'].str.split(',').explode().unique())
len(genres)

659

In [372]:
dataset.fit_partial(item_features=genres)

In [373]:
num_users, num_items = dataset.interactions_shape()
num_users, num_items

(158325, 63758)

In [374]:
lightfm_mapping = dataset.mapping()
lightfm_mapping = {
    'users_mapping': lightfm_mapping[0],
    'user_features_mapping': lightfm_mapping[1],
    'items_mapping': lightfm_mapping[2],
    'item_features_mapping': lightfm_mapping[3],
}
print('users_mapping len - ', len(lightfm_mapping['users_mapping']))
print('user_features_mapping len - ', len(lightfm_mapping['user_features_mapping']))
print('items_mapping len - ', len(lightfm_mapping['items_mapping']))
print('Users item_features_mapping len - ', len(lightfm_mapping['item_features_mapping']))

users_mapping len -  158325
user_features_mapping len -  158335
items_mapping len -  63758
Users item_features_mapping len -  64417


In [375]:
lightfm_mapping['users_inv_mapping'] = {v: k for k, v in lightfm_mapping['users_mapping'].items()}
lightfm_mapping['items_inv_mapping'] = {v: k for k, v in lightfm_mapping['items_mapping'].items()}

In [376]:
num_user_features = dataset.user_features_shape()
num_show_features = dataset.item_features_shape()
print('Num user features: {} -> {}\nnum item features: {} -> {}.'.format(
    num_user_features[1] - num_users, num_user_features[1], 
    num_show_features[1] - num_items, num_show_features[1]))

Num user features: 10 -> 158335
num item features: 659 -> 64417.


In [377]:
def df_to_tuple_iterator(df):
    return zip(*df.values.T)

def concat_last_to_list(t):
    return (t[0], list(t[1:])[0])

def df_to_tuple_list_iterator(df):
    return map(concat_last_to_list, zip(*df.values.T))

In [378]:
train_idx, test_idx, info = folds_with_stats[0]

train = interactions.loc[train_idx]
test = interactions.loc[test_idx]

# popravka

In [379]:
train = interactions

In [380]:
train_mat, train_mat_weights = dataset.build_interactions(df_to_tuple_iterator(train[['user_id', 'item_id']]))
train_mat

<158325x63758 sparse matrix of type '<class 'numpy.int32'>'
	with 1562505 stored elements in COOrdinate format>

In [381]:
train_mat_weights

<158325x63758 sparse matrix of type '<class 'numpy.float32'>'
	with 1562505 stored elements in COOrdinate format>

In [382]:
users['features'] = users[['age', 'sex']].astype(str).apply(lambda x: list(x), axis=1)
users.head()

,user_id,age,sex,features
0,0,45_54,1.0,"[45_54, 1.0]"
1,1,25_34,0.0,"[25_34, 0.0]"
2,2,45_54,0.0,"[45_54, 0.0]"
3,3,65_inf,0.0,"[65_inf, 0.0]"
4,4,18_24,0.0,"[18_24, 0.0]"


In [383]:
known_users_filter = users['user_id'].isin(interactions['user_id'].unique())
train_user_features = dataset.build_user_features(
    df_to_tuple_list_iterator(
        users.loc[known_users_filter, ['user_id', 'features']]
    )
)
train_user_features

<158325x158335 sparse matrix of type '<class 'numpy.float32'>'
	with 419941 stored elements in Compressed Sparse Row format>

In [384]:
items['features'] = items['genres'].str.split(',')
items.head()

,id,title,genres,authors,year,features
0,248031,ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886,"[Зарубежные детские книги, Сказки, Зарубежная ..."
1,256084,скрипка ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894,"[Классическая проза, Литература 19 века, Русск..."
2,134166,испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869,"[Зарубежная классика, Классическая проза, Лите..."
3,281311,странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831,"[Пьесы и драматургия, Литература 19 века]"
4,213473,господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873,"[Зарубежная классика, Классическая проза, Лите..."


In [385]:
items['features'].map(len).value_counts(dropna=False)

2     22291
1     21131
3     14028
4      4458
5      1134
6       419
7       149
8        72
9        33
10       19
11       11
13        7
12        3
17        2
16        1
Name: features, dtype: int64

In [386]:
known_items_filter = items['id'].isin(interactions['item_id'].unique())
train_items_features = dataset.build_item_features(
    df_to_tuple_list_iterator(
        items.loc[known_items_filter, ['id', 'features']]
    )
)
train_items_features

<63758x64417 sparse matrix of type '<class 'numpy.float32'>'
	with 199975 stored elements in Compressed Sparse Row format>

In [387]:
lfm_model = LightFM(no_components=64, learning_rate=0.05, loss='warp', max_sampled=5, random_state=23)
num_epochs = 15
for _ in tqdm(range(num_epochs), total=num_epochs):
    lfm_model.fit_partial(
        train_mat, 
        user_features=train_user_features,
        item_features=train_items_features,
        num_threads=4
    )

In [388]:
lfm_model

In [302]:
top_N = 10
user_id = sample['Id'].iloc[0]
row_id = lightfm_mapping['users_mapping'][user_id]
print(f'Рекомендации для пользователя {user_id}, номер строки - {row_id}')

Рекомендации для пользователя 10001, номер строки - 112297


In [303]:
all_cols = list(lightfm_mapping['items_mapping'].values())
len(all_cols)

63758

In [304]:
pred = lfm_model.predict(row_id, all_cols, user_features=train_user_features, item_features=train_items_features, num_threads=4)
pred, pred.shape

(array([-169.27672, -170.01845, -168.64223, ..., -170.46484, -171.54196,
        -170.57535], dtype=float32),
 (63758,))

In [305]:
top_cols = np.argpartition(pred, -np.arange(top_N))[-top_N:][::-1]
top_cols

array([ 3760,  1760, 32091, 20089, 35075,  3443, 15833,  1915, 29802,
       41123], dtype=int64)

In [306]:
pred[top_cols]

array([-165.03596, -165.49887, -165.91653, -165.92671, -165.94868,
       -166.0678 , -166.10965, -166.13618, -166.26285, -166.28117],
      dtype=float32)

In [307]:
np.max(pred)

-165.03596

In [308]:
recs = pd.DataFrame({'col_id': top_cols})
recs['item_id'] = recs['col_id'].map(lightfm_mapping['items_inv_mapping'].get)
recs['title'] = recs['item_id'].map(item_titles.get)
recs.head()

,col_id,item_id,title
0,3760,37493,снеговик
1,1760,316158,исчезнувшая
2,32091,72970,гончие лилит
3,20089,75731,земное притяжение
4,35075,390687,смертельная белизна


In [311]:
recs = pd.DataFrame({
    'user_id': sample['Id'].unique()
})
recs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3074 entries, 0 to 3073
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  3074 non-null   int64
dtypes: int64(1)
memory usage: 24.1 KB


In [312]:
sample.head()

,Id,Predicted
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...


In [321]:
forrand=pd.read_csv('interactions.csv')

In [389]:
def generate_lightfm_recs_mapper(model, item_ids, known_items, user_features, item_features, N, user_mapping, item_inv_mapping, num_threads=4):
    def _recs_mapper(user):
        try:
            user_id = user_mapping[user]
        except KeyError:
            import random
            user_id =user_mapping[random.choice(list(forrand['user_id']))]
        
        recs = model.predict(user_id, item_ids, user_features=user_features, item_features=item_features, num_threads=num_threads)
        
        additional_N = len(known_items[user_id]) if user_id in known_items else 0
        total_N = N + additional_N
        top_cols = np.argpartition(recs, -np.arange(total_N))[-total_N:][::-1]
        
        final_recs = [item_inv_mapping[item] for item in top_cols]
        if additional_N > 0:
            filter_items = known_items[user_id]
            final_recs = [item for item in final_recs if item not in filter_items]
        return final_recs[:N]
    return _recs_mapper

In [390]:
known_items = train.groupby('user_id')['item_id'].apply(list).to_dict()


In [391]:
mapper = generate_lightfm_recs_mapper(
    lfm_model, 
    item_ids=all_cols, 
    known_items=known_items,
    N=top_N,
    user_features=train_user_features, 
    item_features=train_items_features, 
    user_mapping=lightfm_mapping['users_mapping'],
    item_inv_mapping=lightfm_mapping['items_inv_mapping'],
    num_threads=4
)

In [392]:
%%time
recs['item_id'] = recs['user_id'].map(mapper)

Wall time: 1min 13s


In [393]:
recs.head()

,user_id,item_id
0,10001,"[37493, 316158, 72970, 75731, 390687, 75989, 1..."
1,10002,"[1194, 80576, 139365, 140712, 340191, 55140, 1..."
2,100152,"[249083, 80576, 30755, 354482, 311426, 164675,..."
3,100197,"[354482, 148860, 341481, 319710, 117325, 27690..."
4,100284,"[103707, 389291, 260913, 80576, 81107, 37493, ..."


In [394]:
recs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3074 entries, 0 to 3073
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  3074 non-null   int64 
 1   item_id  3074 non-null   object
dtypes: int64(1), object(1)
memory usage: 48.2+ KB


In [395]:
recs = recs.explode('item_id')
recs['rank'] = recs.groupby('user_id').cumcount() + 1

### Функция для конвертации датафрейма с рекомендациями в нужный формат для сабмита

In [396]:
def make_submission(df, id_col='user_id', predicted_col='item_id'):
    
    df_copy = df.copy()
    df_copy.loc[:, id_col] = df_copy[id_col].astype(str)
    df_copy.loc[:, predicted_col] = df_copy[predicted_col].astype(str)
    df_copy = df_copy.groupby(id_col, as_index=False).agg({predicted_col: lambda x: ' '.join(list(x))})
    df_copy.rename(columns={id_col: 'Id', predicted_col: 'Predicted'}, inplace=True)
    
    return df_copy[['Id', 'Predicted']]

In [397]:
submission_recs = make_submission(recs)
submission_recs.head()

,Id,Predicted
0,10001,37493 316158 72970 75731 390687 75989 175209 2...
1,10002,1194 80576 139365 140712 340191 55140 186002 1...
2,100152,249083 80576 30755 354482 311426 164675 280818...
3,100197,354482 148860 341481 319710 117325 276903 8057...
4,100284,103707 389291 260913 80576 81107 37493 249083 ...


In [400]:
submission_recs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3074 entries, 0 to 3073
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         3074 non-null   object
 1   Predicted  3074 non-null   object
dtypes: object(2)
memory usage: 72.0+ KB


In [399]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3074 entries, 0 to 3073
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         3074 non-null   int64 
 1   Predicted  3074 non-null   object
dtypes: int64(1), object(1)
memory usage: 48.2+ KB


In [403]:
submission_recs.to_csv('submission_recs7.csv', index=False)